In [8]:
import os.path
import pandas as pd

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

SAMPLE_SPREADSHEET_ID = "1gNMdlnevrfawztpJdujWn54WxohZB9pLD5KG5EapImM"
SAMPLE_RANGE_NAME = 'Sheet1!A1:M10' 

CREDENTIALS_PATH = "../credentials.json"
TOKEN_PATH = "../token.json"


def fetch_sheet():
    '''
    Fetches data from the excel sheet and stores returns it as a pandas dataframe
    '''
    
    creds = None

    # Check if token.json exists and load it if it does
    if os.path.exists(TOKEN_PATH):
        creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)

    # If no valid credentials are available, log in again
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())  # Refresh the token if it’s expired
        else:
            # Log in and get new credentials
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_PATH, SCOPES)
            creds = flow.run_local_server(port=0)
        
        # Save the credentials for future runs
        with open(TOKEN_PATH, "w") as token:
            token.write(creds.to_json())

    try:
        # Call the Sheets API
        service = build("sheets", "v4", credentials=creds)
        sheet = service.spreadsheets()
        result = (
            sheet.values()
            .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
            .execute()
        )
        # retrieve the sheet data
        sheet = result.get("values", [])
        
        # Check if there's data in the sheet
        if not sheet:
            return "No data found in the Google Sheet."

        # Here the sheet is a pandas dataframe
        df = pd.DataFrame(sheet[1:], columns=sheet[0])
        
        return df

    except HttpError as err:
        return f"An error occurred: {err}"

def get_claim_status(df, id):
    try:
        status_msg = df[df["Claim ID"] == id]["Approval Status"].values[0]
    except IndexError:
        return {"error": True, "status_msg": "Please Enter a correct index"}
    
    return {"error": False, "status_msg": status_msg}

data = fetch_sheet()


In [9]:
data.head()

,Claim ID,Department,Claimant Name,Date of Claim,Expense Category,Amount Claimed,Approval Status,Receipt (Link/Yes/No),Description of Expense,Date Approved,Amount Approved,Date Reimbursed,Remarks
0,1,Publicity,Berry,01/09/2024,Advertising,$500,Pending,Yes,Social media campaign,N/A,N/A,N/A,N/A
1,2,Admin,Jerry,02/09/2024,Stationery,$100,Approved,Yes,Office supplies,03/09/2024,$100,04/09/2024,N/A
2,3,Logistics,Kailin,03/09/2024,Transport,$200,Rejected,Yes,Van rental,N/A,N/A,N/A,Insufficient details
3,4,First Aid,Sean,03/09/2024,Medical supplies,$150,Approved,Yes,First aid kits,04/09/2024,$150,05/09/2024,N/A
4,5,Admin,Berry,02/09/2024,Stationery,$100,Approved,Yes,Office supplies,03/09/2024,$100,04/09/2024,N/A


In [13]:
department = "Admin"
id = "uuid generated 1"

new_row = pd.DataFrame({'Department': ["efef"],
                        'Claimant Name': ["jefef"],
                        'Date of Claim': ["eefe"]})

# Concatenate the original DataFrame with the new row
data = pd.concat([data, new_row], ignore_index=True)

In [14]:
data

,Claim ID,Department,Claimant Name,Date of Claim,Expense Category,Amount Claimed,Approval Status,Receipt (Link/Yes/No),Description of Expense,Date Approved,Amount Approved,Date Reimbursed,Remarks
0,1,Publicity,Berry,01/09/2024,Advertising,$500,Pending,Yes,Social media campaign,N/A,N/A,N/A,N/A
1,2,Admin,Jerry,02/09/2024,Stationery,$100,Approved,Yes,Office supplies,03/09/2024,$100,04/09/2024,N/A
2,3,Logistics,Kailin,03/09/2024,Transport,$200,Rejected,Yes,Van rental,N/A,N/A,N/A,Insufficient details
3,4,First Aid,Sean,03/09/2024,Medical supplies,$150,Approved,Yes,First aid kits,04/09/2024,$150,05/09/2024,N/A
4,5,Admin,Berry,02/09/2024,Stationery,$100,Approved,Yes,Office supplies,03/09/2024,$100,04/09/2024,N/A
5,6,Logistics,Jerry,03/09/2024,Transport,$200,Rejected,Yes,Van rental,N/A,N/A,N/A,Insufficient details
6,7,First Aid,Kailin,03/09/2024,Medical supplies,$150,Approved,Yes,First aid kits,04/09/2024,$150,05/09/2024,N/A
7,8,Admin,Sean,02/09/2024,Stationery,$100,Approved,Yes,Office supplies,03/09/2024,$100,04/09/2024,N/A
8,9,Logistics,Berry,03/09/2024,Transport,$200,Rejected,Yes,Van rental,N/A,N/A,N/A,Insufficient details
9,NaN,efef,jefef,eefe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
data.columns

Index(['Claim ID', 'Department', 'Claimant Name', 'Date of Claim',
       'Expense Category', 'Amount Claimed', 'Approval Status',
       'Receipt (Link/Yes/No)', 'Description of Expense', 'Date Approved',
       'Amount Approved', 'Date Reimbursed', 'Remarks'],
      dtype='object')

In [18]:
x = {'spreadsheetId': '1gNMdlnevrfawztpJdujWn54WxohZB9pLD5KG5EapImM', 'tableRange': 'Sheet1!A1:M12', 'updates': {'spreadsheetId': '1gNMdlnevrfawztpJdujWn54WxohZB9pLD5KG5EapImM', 'updatedRange': 'Sheet1!A13:G13', 'updatedRows': 1, 'updatedColumns': 7, 'updatedCells': 7}}
x['updates']


{'spreadsheetId': '1gNMdlnevrfawztpJdujWn54WxohZB9pLD5KG5EapImM',
 'updatedRange': 'Sheet1!A13:G13',
 'updatedRows': 1,
 'updatedColumns': 7,
 'updatedCells': 7}